In [1]:
import pandas as pd
import numpy as np
from darts import TimeSeries
from darts.models import LightGBMModel
import lightgbm as lgb

In [71]:
import pandas as pd
import numpy as np
from darts import TimeSeries
from darts.models import LightGBMModel
import lightgbm as lgb

# Load ais_train.csv with separator '|'
train_df = pd.read_csv('ais_train.csv', sep='|')
train_df['time'] = pd.to_datetime(train_df['time'])

# Load ais_test.csv with separator ','
test_df = pd.read_csv('ais_test.csv', sep=',')
test_df['time'] = pd.to_datetime(test_df['time'])

# Use 'vesselId' instead of 'vessel_id'
# Select only vessel IDs that are in both train and test datasets
common_vessel_ids = set(train_df['vesselId']).intersection(set(test_df['vesselId']))
train_df = train_df[train_df['vesselId'].isin(common_vessel_ids)]

###

if False:    
    # Sort the training data by vesselId and time
    train_df = train_df.sort_values(['vesselId', 'time'])
    
    # Shift latitude and longitude to create Ylatitude and Ylongitude
    train_df['Ylatitude'] = train_df.groupby('vesselId')['latitude'].shift(-1)
    train_df['Ylongitude'] = train_df.groupby('vesselId')['longitude'].shift(-1)
    
    # Calculate time difference between consecutive records
    train_df['time_diff'] = train_df.groupby('vesselId')['time'].shift(-1) - train_df['time']
    train_df['time_diff'] = train_df['time_diff'].dt.total_seconds() / 3600  # Convert to hours
    
    # Drop the last record of each vesselId group since it will have NaN values after shifting
    train_df = train_df.dropna(subset=['Ylatitude', 'Ylongitude', 'time_diff'])

##

# Group the training data by vesselId
groups = train_df.groupby('vesselId')

# Initialize dictionaries to store TimeSeries objects and last training times
timeseries_dict = {}
last_train_time = {}

# Process each vesselId group
for vessel_id, group_df in groups:
    # Sort on time
    group_df = group_df.sort_values('time')
    # Set index to time
    group_df = group_df.set_index('time')
    # Select features (latitude and longitude)
    features_df = group_df[['latitude', 'longitude']]
    # Resample data to hourly frequency with mean and linear interpolation
    features_df = features_df.resample('h').mean().interpolate(method='cubic')
    # Create Darts TimeSeries object
    ts = TimeSeries.from_dataframe(features_df, value_cols=['latitude', 'longitude'])
    # Store the TimeSeries object and last training time
    timeseries_dict[vessel_id] = ts
    last_train_time[vessel_id] = features_df.index.max()

# Initialize a dictionary to store predictions
predictions = {}

# Fit LightGBM models and predict for each TimeSeries object
for vessel_id, ts in timeseries_dict.items():
    # Get the last training time
    last_time = last_train_time[vessel_id]
    # Get test times for this vessel
    vessel_test_df = test_df[test_df['vesselId'] == vessel_id]
    test_times = vessel_test_df['time']
    # Compute the time differences in hours
    time_diffs = (test_times - last_time).dt.total_seconds() / 3600
    # Get the maximum forecast horizon needed
    max_n = int(np.ceil(time_diffs.max()))
    if max_n <= 0:
        continue  # Skip if no future times to predict
    # Initialize LightGBM model with lag parameters
    model = LightGBMModel(
        lags=48, # the correct value is between 48 and 96
        learning_rate=0.1,
        #n_estimators=100,
        #max_depth=10,
    )
    # Fit the model
    model.fit(ts)
    # Predict up to the maximum horizon needed
    forecast = model.predict(max_n)
    # Store the forecast and last time
    predictions[vessel_id] = (forecast, last_time)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002364 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 48960
[LightGBM] [Info] Number of data points in the train set: 2976, number of used features: 192
[LightGBM] [Info] Start training from score 44.043567
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002199 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 48960
[LightGBM] [Info] Number of data points in the train set: 2976, number of used features: 192
[LightGBM] [Info] Start training from score -4.783110
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002471 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 48960
[LightGBM] [Info] Number of data points in the train set: 2939, number of used features: 192
[LightGBM] [Info] Star

In [72]:
# Initialize a list to store submission rows
submission_rows = []

# Generate predictions for the submission file
for idx, row in test_df.iterrows():
    vessel_id = row['vesselId']
    test_time = row['time']
    test_id = row['ID']  # Assuming 'ID' column exists in test_df
    # Check if predictions are available for this vessel_id
    if vessel_id in predictions:
        forecast_ts, last_time = predictions[vessel_id]
        time_diff = (test_time - last_time).total_seconds() / 3600
        index = int(np.round(time_diff)) - 1  # Adjust index since forecast starts from last_time + 1 hour
        # Convert forecast_ts to DataFrame
        forecast_df = forecast_ts.pd_dataframe()
        # Check if index is within forecast horizon
        if 0 <= index < len(forecast_df):
            predicted_lat = forecast_df['latitude'].iloc[index]
            predicted_lon = forecast_df['longitude'].iloc[index]
        else:
            predicted_lat = np.nan
            predicted_lon = np.nan
    else:
        predicted_lat = np.nan
        predicted_lon = np.nan
    # Append the prediction to the submission list
    submission_rows.append({
        'ID': test_id,
        'longitude_predicted': predicted_lon,
        'latitude_predicted': predicted_lat
    })

# Create a submission DataFrame from the list
submission_df = pd.DataFrame(submission_rows)

# Save the submission file
submission_df.to_csv('submission.csv', index=False)

print(submission_df)


          ID  longitude_predicted  latitude_predicted
0          0           -81.478133           31.348673
1          1           120.317468           14.818287
2          2            10.815008           38.428888
3          3           172.848514          -43.530992
4          4            -6.328304           48.372523
...      ...                  ...                 ...
51734  51734           -79.424994           27.023285
51735  51735           143.283478           41.509353
51736  51736           141.826772           35.360611
51737  51737            13.540887           55.539451
51738  51738             9.539468           55.840229

[51739 rows x 3 columns]


In [73]:
submission_df.describe()

,ID,longitude_predicted,latitude_predicted
count,51739.000000,51739.000000,51739.000000
mean,25869.000000,9.370142,38.826012
std,14935.907126,56.711245,21.923283
min,0.000000,-122.877293,-43.530992
25%,12934.500000,-5.200930,36.044999
50%,25869.000000,4.821660,43.568404
75%,38803.500000,13.097562,52.891986
max,51738.000000,175.397128,63.609159
